In [2]:
import pandas as pd
import scipy.io
import scipy.signal as ss
import pywt
import glob
import matplotlib.pyplot as plt
import scaleogram as scg
from scaleogram import cws
import numpy as np
from math import *
import os

In [3]:
print(os.getcwd())
os.chdir("..")

e:\git\HeartResearch\Data Preprocessing


In [4]:
main_data_dir = os.getcwd() + "\Data set"
print(os.listdir(main_data_dir))

['Label.csv', 'Median_Filter', 'Output', 'Output.png', 'README.md', 'Scaleogram_512_256', 'TrainingSet1', 'TrainingSet2', 'TrainingSet3']


In [5]:
data_mat_files = glob.glob(main_data_dir + "/TrainingSet*/*")
label_path = main_data_dir + "/Label.csv"
label_df = pd.read_csv(label_path)

In [6]:
def data_extract(signal_raw_list, min_len = 3000):
    signal_list = []
    signal_number_list = []
    
    for signal in signal_raw_list:
        mat = scipy.io.loadmat(signal)
        signal_list.append(mat['ECG'][0][0][2][:,:min_len])
        signal_number = signal.split('\\')[-1].split('.')[0]
        signal_number_list.append(signal_number)

    return (np.array(signal_list), signal_number_list)

signal_list, signal_number = data_extract(data_mat_files)
print(signal_list.shape)

(6877, 12, 3000)


In [6]:
def median_filter(signal_raw_list):
    median_fil_list = []

    for signal in signal_raw_list:
        median_fil = ss.medfilt(signal)
        median_fil_list.append(median_fil)    
    return median_fil_list
    
median_fil_list = median_filter(signal_list)

In [22]:
def save_data_filter(data, signal_number, folder):
    for i in range(len(data)):
        save_dic_data = {"ECG": data[i]}
        scipy.io.savemat("E:\git\HeartResearch\Data set\{}\{}.mat".format(folder,signal_number[i]), save_dic_data)

In [8]:
folder = "Median_Filter"
save_data_filter(median_fil_list, signal_number)

In [9]:
output_path = "E:\git\HeartResearch\Data set\Scaleogram_512_256\Median_Filter\{}_{}"
scg.set_default_wavelet('morl')
px = 1/plt.rcParams['figure.dpi']
lead = ['I','II','III','aVR','aVL','aVF','V1','V2','V3','V4','V5','V6']

def scaleogram(data):
    for i in range(len(data)):
        for j in range(len(lead)):

            signal_length = 400
            # range of scales to perform the transform
            scales = scg.periods2scales(np.arange(1, signal_length+1))
            x_values_wvt_arr = range(0,len(data[i][j]),1)

            # plot the signal 
            fig1, ax1 = plt.subplots(1, 1, figsize=(9, 3.5));  
            ax1.plot(x_values_wvt_arr, data[i][j], linewidth=3, color='blue')
            ax1.set_xlim(0, signal_length)
            # ax1.set_title("ECG")

            # the scaleogram
            fig = cws(data[i][j][:signal_length], scales=scales, figsize=(512*px, 256*px), coi = False)
            print(fig)
            plt.axis('off')
            plt.savefig(output_path.format(signal_number[i],lead[j]), bbox_inches='tight', pad_inches = 0)

In [ ]:
scaleogram(median_fil_list[:1])

In [7]:
def gaussian_filter(signal_raw_list, std):
    gauss_filt = []
    n_bins = signal_raw_list.shape[2]
    M = n_bins - 1 if n_bins % 2 == 0 else n_bins
    window = ss.gaussian(M, std = std)

    for signal in signal_raw_list:
        for i in range(signal.shape[0]):
            gauss_filter = ss.convolve(signal[i], window, mode='same', method='auto')
            gauss_filt.append(gauss_filter)    
    return gauss_filt

In [8]:
gauss_filt = gaussian_filter(signal_list, std=1)

In [17]:
gauss_filt_array = (np.array(gauss_filt)).reshape(6877,12,3000)
gauss_filt_list = []
for i in range(gauss_filt_array.shape[0]):
    gauss_filt_list.append(gauss_filt_array[i])

In [23]:
folder = "Gaussian_Filter"
save_data_filter(gauss_filt_list, signal_number, folder)